In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pysolar.solar import get_azimuth, get_altitude
from pytz import timezone
import ipywidgets as widgets
from ipywidgets import interact, interactive

In [2]:
def getSolarPosition(latitude: float = -0.2105367, longitude: float = -78.491614, date: datetime = datetime.now(tz=timezone("America/Guayaquil"))):
    """Calcula el azimuth y la elevación para una posición geográfica y fecha."""
    az = get_azimuth(latitude, longitude, date)
    el = get_altitude(latitude, longitude, date)
    return az, el

In [3]:
def calculateControlAngles(azimuth, elevation):
    """Calcula los ángulos de control (pitch y roll) en base a la posición solar."""
    # Convertir los ángulos a radianes
    elevation_rad = math.radians(elevation)
    azimuth_rad = math.radians(azimuth)

    # Definir las componentes del vector solar
    S_x = math.cos(elevation_rad) * math.sin(azimuth_rad)
    S_y = math.cos(elevation_rad) * math.cos(azimuth_rad)
    S_z = math.sin(elevation_rad)

    # Calcular roll (φ) y pitch (β) usando las ecuaciones matemáticas
    if S_y != 0:
        roll = math.degrees(math.atan2(S_y, S_x))
    else:
        roll = 0
    
    pitch = math.degrees(math.asin(S_z))

    return pitch, roll

In [4]:
def plotSunAndPanelTrajectory(start_date, duration_hours, latitude=-0.2105367, longitude=-78.491614):
    """Dibuja la trayectoria del sol y del panel solar de manera interactiva."""
    # Convertir la fecha a timezone-aware si no lo es
    if start_date.tzinfo is None:
        start_date = timezone("America/Guayaquil").localize(start_date)

    end_date = start_date + timedelta(hours=duration_hours)
    times = [start_date + timedelta(minutes=10*i) for i in range(int((end_date - start_date).total_seconds() / 600))]
    azimuths, elevations, pitches, rolls = [], [], [], []

    for time in times:
        azimuth, elevation = getSolarPosition(latitude, longitude, time)
        pitch, roll = calculateControlAngles(azimuth, elevation)
        azimuths.append(azimuth)
        elevations.append(elevation)
        pitches.append(pitch)
        rolls.append(roll)
    
    # Convertir azimut y elevación a coordenadas 3D
    sun_x = np.cos(np.radians(elevations)) * np.sin(np.radians(azimuths))
    sun_y = np.cos(np.radians(elevations)) * np.cos(np.radians(azimuths))
    sun_z = np.sin(np.radians(elevations))

    # Convertir pitch y roll a coordenadas 3D
    panel_x = np.cos(np.radians(pitches)) * np.sin(np.radians(rolls))
    panel_y = np.cos(np.radians(pitches)) * np.cos(np.radians(rolls))
    panel_z = np.sin(np.radians(pitches))

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(sun_x, sun_y, sun_z, label='Trayectoria del Sol', color='orange')
    ax.plot(panel_x, panel_y, panel_z, label='Orientación del Panel', color='blue')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Trayectoria del Sol y Orientación del Panel Solar\n{start_date.date()}')
    ax.legend()
    plt.show()

In [5]:
def interactive_plot():
    """Crea una interfaz interactiva para ingresar la fecha y duración de la simulación."""
    date_picker = widgets.DatePicker(description='Fecha:', disabled=False)
    time_picker = widgets.FloatSlider(value=6, min=0, max=23, step=1, description='Hora Inicial:')
    duration_picker = widgets.FloatSlider(value=12, min=1, max=24, step=1, description='Duración (h):')

    def update_plot(date, start_hour, duration):
        start_datetime = datetime.combine(date, datetime.min.time()) + timedelta(hours=start_hour)
        # Convertir a timezone-aware
        start_datetime = timezone("America/Guayaquil").localize(start_datetime)
        plotSunAndPanelTrajectory(start_datetime, duration)

    interact = widgets.interactive(update_plot, date=date_picker, start_hour=time_picker, duration=duration_picker)
    display(interact)

if __name__ == "__main__":
    interactive_plot()

interactive(children=(DatePicker(value=None, description='Fecha:', step=1), FloatSlider(value=6.0, description…